In [1]:
# Import library-library
import os
import glob
import string

# Data Preparation and Preprocessing
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Word Embedding
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse

from sklearn.metrics.pairwise import cosine_similarity

# Input and Expansion Query
from textblob import TextBlob
from nltk.tokenize import wordpunct_tokenize
#from googletrans import Translator

#Split to train and test
from sklearn.model_selection import train_test_split

import mysql.connector
from mysql.connector import errorcode

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\RayRoyy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
excluded_words = ["tempat", "waktu"]

NLTK_StopWords = stopwords.words('indonesian')
NLTK_StopWords.extend(["detik", "detikjatim", "detikjateng", "detikjabar", "detiksulsel", "detiksumbar", "detikbali", "detikpapua", "detiksulteng", "detikmaluku", "detjatim", "detikcom", "allahumma", "aamiin", "aamiin", "allah", "bismillah"])
NLTK_StopWords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah'])
txt_stopword = pd.read_csv("stopwords.txt", names= ["stopwords"], header = None)

# convert stopword string to list & append additional stopword
NLTK_StopWords.extend(txt_stopword["stopwords"][0].split(' '))
NLTK_StopWords = set(NLTK_StopWords)

In [3]:
def preprocessing(berita):
    # Preprocessing
    s = str(berita)
    s = s.lower()
    s = s.replace('\n', ' ')
    s = s.replace('\r', ' ')
    s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
    tokens = [token for token in s.split(" ") if token != ""]

    # Stopwords checking
    T = [t for t in tokens if ((t in excluded_words) or (t not in NLTK_StopWords))]
    return T

In [4]:
# df =pd.read_csv("news.csv")
# df.info()
# df.head()
dataframe = []

try:
        cnx = mysql.connector.connect(user = 'root', password='Password', database = 'Petakabar')
        cursor = cnx.cursor()

        ambildata = ("SELECT berita_title, berita_date, berita_desc, berita_source FROM berita where berita_topik_id = 4 and class_classification is not null")
        cursor.execute(ambildata)
        dataa = cursor.fetchall()
        for row in dataa:
            dataframe.append(row)
        
except mysql.connector.Error as err:
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print("Something is wrong with your user name or password")
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            print("Database does not exist")
        else:
            print(err)
else:
    cursor.close()
    cnx.close()

In [5]:
import pandas as pd
writer = pd.DataFrame(dataframe, columns=[
                    'Title','Date','Description', 'Source'], index=None)
writer.to_csv('news.csv', index=False, sep=',')

In [6]:
df =pd.read_csv("news.csv")
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1223 entries, 0 to 1222
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        1223 non-null   object
 1   Date         1223 non-null   object
 2   Description  1223 non-null   object
 3   Source       1223 non-null   object
dtypes: object(4)
memory usage: 38.3+ KB


,Title,Date,Description,Source
0,"Ramalan Zodiak Aries Hari Ini, 2 Desember 2022...","Jumat, 02 Des 2022 08:35 WIB","\- Simak ramalan zodiak Aries hari ini, 2 Dese...",www.tribunnews.com
1,"Ramalan Zodiak Sagitarius Besok, 6 Desember 20...","Senin, 05 Des 2022 14:55 WIB",\- Inilah ramalan zodiak Sagitarius untuk beso...,www.tribunnews.com
2,"Ramalan Zodiak Gemini Besok, 29 November 2022:...","Senin, 28 Nov 2022 12:52 WIB","\- Inilah ramalan zodiak Gemini besok, 29 Nove...",www.tribunnews.com
3,"Lindungi Diri dan Keluarga, Liburan Tanpa Curiga!","Sabtu, 10 Des 2022 05:59 WIB",Liburan akhir tahun menjadi salah satu kesemp...,www.tribunnews.com
4,Manfaat Jeruk untuk Kesehatan: Turunkan Kolest...,"Rabu, 07 Des 2022 13:40 WIB",\- Inilah sejumlah manfaat jeruk untuk kesehat...,www.tribunnews.com


In [7]:
column = ['Title', 'Date', 'Description', 'Source']
df_total = pd.DataFrame()

df = pd.read_csv("news.csv")
df = df.dropna(subset=['Description'], axis=0)
df.sort_values("Description", inplace = True)
df.drop_duplicates(subset ="Description", keep = False, inplace = True)

df.head()
df_train_unprocessed, df_test_unprocessed = train_test_split(df,test_size=0.1)

In [8]:
# df_first_1000 = df.head(1000)
# len(df_first_1000)
df_train, df_test = train_test_split(df,test_size=0.1)
len(df_train)


1100

In [9]:
df_train.to_csv("corpus/df_train_unprerocessed.csv" , index = None, header=True)
df_test.to_csv("corpus/df_test_unprerocessed.csv" , index = None, header=True)

In [10]:
desc_text_test_unprocessed = []

for x in range(0, df_test_unprocessed.shape[0]):
  desc_text_test_unprocessed.append(df_test_unprocessed.iloc[x, -2])

joblib.dump(desc_text_test_unprocessed, "corpus/model/desc_text_test_unprocessed.pkl")

['corpus/model/desc_text_test_unprocessed.pkl']

In [11]:
# for x in range(0, df.shape[0]):
#     text = preprocessing(df.iloc[x, -2])
#     df.iloc[x, -2] = (' '.join(text)) 

desc_text_train_unpreprocessed = []
desc_text_test_unpreprocessed = []

for x in range(0, df_train.shape[0]):
  desc_text_train_unpreprocessed.append(df_train.iloc[x, -2])

for x in range(0, df_test.shape[0]):
  desc_text_test_unpreprocessed.append(df_test.iloc[x, -2])

In [12]:
# Simpan teks asli dokumen
joblib.dump(desc_text_test_unpreprocessed, "corpus/model/desc_text_test_unpreprocessed.pkl")
joblib.dump(desc_text_train_unpreprocessed, "corpus/model/desc_text_train_unpreprocessed.pkl")

# Simpan dataframe hasil text processing sebelum TF-IDF Vectorizer
df.to_csv('corpus/dataset/df_total_unpreprocessed.csv', index=False)
df_test.to_csv('corpus/dataset/df_test_unpreprocessed.csv', index=False)
df_train.to_csv('corpus/dataset/df_train_unpreprocessed.csv', index=False)


In [13]:
for x in range(0, df.shape[0]):
    text = preprocessing(df.iloc[x, -2])
    df.iloc[x, -2] = (' '.join(text)) 

In [14]:
desc_text_train = []
desc_text_test = []

for x in range(0, df_train.shape[0]):
  desc_text_train.append(df_train.iloc[x, -2])

for x in range(0, df_test.shape[0]):
  desc_text_test.append(df_test.iloc[x, -2])

In [15]:
# print(df_test.info())
print("Panjang Dokumen Test : ", len(desc_text_test))

print ('-'*90)

# print(df_train.info())
print("Panjang Dokumen Train : ", len(desc_text_train))

Panjang Dokumen Test :  123
------------------------------------------------------------------------------------------
Panjang Dokumen Train :  1100


In [16]:
# Simpan teks asli dokumen
joblib.dump(desc_text_test, "corpus/model/desc_text_test.pkl")
joblib.dump(desc_text_train, "corpus/model/desc_text_train_new.pkl") #INI

# Simpan dataframe hasil text processing sebelum TF-IDF Vectorizer
df.to_csv('corpus/dataset/df_total_new.csv', index=False) #INI
df_test.to_csv('corpus/dataset/df_test.csv', index=False)
df_train.to_csv('corpus/dataset/df_train.csv', index=False)


In [17]:
# Fit transform menggunakan TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_test_matrix = tfidf_vectorizer.fit_transform(desc_text_test)
tfidf_train_matrix = tfidf_vectorizer.fit_transform(desc_text_train)

print('tfidf_test_matrix')
print(tfidf_test_matrix.shape)
print('tfidf_train_matrix')
print(tfidf_train_matrix.shape)

tfidf_test_matrix
(123, 8450)
tfidf_train_matrix
(1100, 26454)


In [18]:
joblib.dump(tfidf_vectorizer, "corpus/vectorizer/vectorizer_new.pkl") #INI
joblib.dump(tfidf_test_matrix, "corpus/matrix/tfidf_test.pkl")
joblib.dump(tfidf_train_matrix, "corpus/matrix/tfidf_train_new.pkl") #INI

['corpus/matrix/tfidf_train_new.pkl']

### Pencarian 

In [19]:
df_total = pd.read_csv('corpus/dataset/df_total_new.csv')
df_total = df_total[pd.notnull(df_total['Description'])]
print(df_total.info())
print ('-'*90)

document_text_train= joblib.load('corpus/model/desc_text_train_new.pkl')
document_text_test= joblib.load('corpus/model/desc_text_test.pkl')
print(len(document_text_test))
print(len(document_text_train))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1223 entries, 0 to 1222
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        1223 non-null   object
 1   Date         1223 non-null   object
 2   Description  1223 non-null   object
 3   Source       1223 non-null   object
dtypes: object(4)
memory usage: 47.8+ KB
None
------------------------------------------------------------------------------------------
123
1100


In [22]:
kueri='penyakit'
kueri=preprocessing(kueri)
kueri= [" ".join (kueri)]
print (kueri)

tfidf_matrix =joblib.load( "corpus/matrix/tfidf_train_new.pkl" )
tfidf_vectorizer = joblib.load( "corpus/vectorizer/vectorizer_new.pkl" ) 
query_vec= tfidf_vectorizer.transform(kueri)
results=cosine_similarity(tfidf_matrix, query_vec).reshape((-1))
#print (results)
print("\n======================\n")
print("Top 10 most similar documents in corpus:")

j = 1

for i in results.argsort()[-10:][::-1]:
    print("No ID Dokumen  : ", i)
    print("Tanggal        : ", df_total.iloc[i,1])
    print("Isi berita     : ", df_total.iloc[i,2])
    print("(Score: %.4f) " % results[i])

['penyakit']


Top 10 most similar documents in corpus:
No ID Dokumen  :  234
Tanggal        :  Jumat, 09 Des 2022 10:41 WIB
Isi berita     :  penumpang bernama andari menuliskan pesan menyentuh pengalamannya kereta rel listrik krl depok waktu cerita dibenarkan vice president corporate secretary kai commuter erni sylviane purba erni sylviane purba cerita anne dikonfirmasi kompas com jumat 9 12 2022 kendati anne peristiwa detail keterangan resmi perusahaan pesan andari menuliskan pesan terima kasih kereta commuter indonesia layanan sepenuh hati andari mengaku mengalami sakit kepala foto foto siang berbaring krl depok andari tulis andari pesan berjejaring diterima kompas com dikutip jumat 9 12 2022 pesan andari mengaku berkisah berkisah layanan kereta commuter indonesia kci menyentuh hati siang andari pulang rumah sakit cipto mangunkusumo rscm salemba jakart pusat tindakan medis penyakit derita stasiun cikini andari merasakan nyeri hebat muncul kepala kanan andari membawa bantal pagi ber